# Upload masks to a BrainBox project

Katja, spring 2021

## Imports

In [4]:
import urllib.request as urlreq
import re
import os
import numpy as np
import pandas as pd
import scipy as sp

import json

In [ ]:
#!pip install joblib

## Define paths and subjects

In [1]:
# local path to segmentations
SEG_DIR = '../data/derived/seg'

In [11]:
# subject ID --> if the BrainBox project has a specified column for subjectID, subjectID can be queried from there (note: the "Name" field which is automatically populated from the csv file cannot be queried)
# if not, we have to define the list as a string here
subId = ['sub1', 'sub2', 'sub3']


In [12]:
print(len(subId))

3


## Upload segmentations to BrainBox

The example shows placeholders for the imaginary project https://brainbox.pasteur.fr/project/yourproject

In [62]:
# in case your project is a closed project, you will need tto get a token: 
# Get your token by logging into BrainBox and obtain your token at https://brainbox.pasteur.fr/mri/upload
# token valid for 24h
token = "5p6g167929k6vsuuyp4f59"

# Get data from your BrainBox Project
prj_url = "http://brainbox.pasteur.fr/project/json/yourproject?token="+token
res = urlreq.urlopen( prj_url )
txt = res.read()
prj = json.loads( txt )
nfiles = len( prj['files']['list'] )

N = nfiles

print(N) 

156


In [72]:
#print ( prj['files']['list'] )
links = prj['files']['list']

In [10]:
links

['https://linkToMyFile/anatomy_1.nii.gz',
 'https://linkToMyFile/anatomy_2.nii.gz',
 'https://linkToMyFile/anatomy_3.nii.gz']

In [ ]:
%%bash
# 2 links for testing
links=('https://linkToMyFile/anatomy_1.nii.gz' 'https://linkToMyFile/anatomy_2.nii.gz')
# links to all anatomical files in the project
#links=('https://linkToMyFile/anatomy_1.nii.gz' 'https://linkToMyFile/anatomy_2.nii.gz' 'https://linkToMyFile/anatomy_3.nii.gz')

# 2 subjects for testing
subjects=('sub1_mask' 'sub2_mask')
# all subjects
#subjects=('sub1_mask' 'sub2_mask' 'sub3_mask')


# path to local segmentation files
pathToSeg=/full/path/to/data/derived/seg


i=0
for link in ${links[@]}; do
    #echo index $i

    sub=${subjects[i]}
    echo $sub
    
    # in case you need to cut part of the naming
    subID=$(printf $sub|cut -d '_' -f 1)
    #echo $subID

    i=$(( i + 1 ))
    #echo index $i
    
    maskPath=${pathToSeg}/sub-${subID}
    
    mask=sub-${subID}.seg.nii.gz
    #echo $mask
    
    # you will have to specify the annotation layer of your BrainBox project, in which you want to upload all segmentations
    # For this, go to your project settings at https://brainbox.pasteur.fr/project/yourproject/settings and 
        # get the name of the annotation layer from there; or create a new layer by clicking the plus `+` icon, 
        # give it a name, type 'volume', and specify the labelset which you would like to use for this annotation layer
    
    curl -F "url=${link}" -F "atlasName=yourannotationlayer" -F "atlasProject=yourproject" -F "atlasLabelSet=freestyle.json" -F "atlas=@${maskPath}/${mask}" -F "token=5p6g167929k6vsuuyp4f59" https://brainbox.pasteur.fr/mri/upload

done